# Imports

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

---

# Carga de datos

In [14]:
features = pd.read_pickle("../../features/auctions_train_ventana_1.pkl")
targets = pd.read_pickle("../../targets/targets_ventana_1_auctions.pkl")
train = features.merge(targets, left_index=True, right_index=True)

---

# Preparación del set de datos para XGBoost

In [64]:
X, y = train.iloc[:,:-1], train.iloc[:,-1]
data_dmatrix = xgb.DMatrix(data=X,label=y)
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [65]:
xg_reg = xgb.XGBRegressor(colsample_bytree = 0.7, learning_rate = 1,
                n_estimators = 100, n_jobs=6, max_depth=7, objective="reg:linear")

In [66]:
xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, gamma=0, learning_rate=1, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=6, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [67]:
preds = xg_reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 93545.206149


---

# Pruebas

In [49]:
features_auctions = pd.read_pickle("../../features/auctions_train_ventana_4.pkl")
features_clicks = pd.read_pickle("../../features/clicks_train_ventana_4.pkl")

In [50]:
features_combinados_imputados = features_auctions.merge(features_clicks, how="left", right_index=True, left_index=True, suffixes=('_auctions', '_clicks')).fillna(0)

In [51]:
#features_auctions = pd.read_pickle("../../features/auctions_train_ventana_1.pkl")
targets = pd.read_pickle("../../targets/targets_ventana_1_auctions.pkl")
train = features_combinados_imputados.merge(targets, left_index=True, right_index=True)

X, y = train.iloc[:,:-1], train.iloc[:,-1]
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

dtrain = xgb.DMatrix(data=X_train, label=y_train)
dtest = xgb.DMatrix(data=X_test, label=y_test)

param = {'max_depth':6,
         'eta':0.1,
         'silent':1,
         'objective':'reg:linear',
         'eval_metric' : 'rmse',
         'obj' : 'reg:squarederror',
         'colsample_bytree' : 0.7,
         'min_child_weight' : 0.5,
         'random_state' : 0,
         'reg_alpha' : 0,
         'reg_lambda' : 1,
         'scale_pos_weight' : 1,
         'learning_rate' : 0.03}


num_round = 5000
stopping = 20
bst = xgb.train(param, dtrain, num_round, evals=[(dtrain, "train"), (dtest, "test")], early_stopping_rounds=stopping)
# make prediction
#preds = bst.predict(dtest)
#Stopping. Best iteration:
#[360]	train-rmse:67651	test-rmse:69665.3

[0]	train-rmse:65893	test-rmse:67000.4
Multiple eval metrics have been passed: 'test-rmse' will be used for early stopping.

Will train until test-rmse hasn't improved in 20 rounds.
[1]	train-rmse:64172.4	test-rmse:65259
[2]	train-rmse:62769.4	test-rmse:63823.2
[3]	train-rmse:61089.4	test-rmse:62112.5
[4]	train-rmse:59470.3	test-rmse:60450.7
[5]	train-rmse:57900	test-rmse:58825.4
[6]	train-rmse:56355.7	test-rmse:57250.4
[7]	train-rmse:55162.6	test-rmse:56013.4
[8]	train-rmse:54016	test-rmse:54815.2
[9]	train-rmse:53170.1	test-rmse:53956.2
[10]	train-rmse:51778.7	test-rmse:52536.3
[11]	train-rmse:50442.5	test-rmse:51155.3
[12]	train-rmse:49154.4	test-rmse:49845.1
[13]	train-rmse:47889.7	test-rmse:48554.2
[14]	train-rmse:47052.1	test-rmse:47712
[15]	train-rmse:45863.1	test-rmse:46500.5
[16]	train-rmse:44968.7	test-rmse:45591.5
[17]	train-rmse:43841.8	test-rmse:44441.1
[18]	train-rmse:42755.7	test-rmse:43334.9
[19]	train-rmse:41703.6	test-rmse:42261
[20]	train-rmse:40691.5	test-rmse:41225

---

In [52]:
import lightgbm as lgb
#lgb_model = lgb.LGBMRegressor(boosting_type='dart', metric='rmse', n_estimators=50, colsample_bytree=0.9)
lgb_model = lgb.LGBMRegressor(boosting_type='dart', num_leaves=7, n_estimators=1000, metric='rmse',
                               learning_rate=0.05, colsample_bytree=0.8)
lgb_model.fit(X_train, y_train)
preds = lgb_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 16291.329651


In [53]:
preds_xgb = bst.predict(dtest)
preds_lgbm = lgb_model.predict(X_test)

In [54]:
p1 = 0.8
p2 = 0.2

preds = p1*preds_xgb + p2*preds_lgbm
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 14170.942906


In [56]:
preds_xgb.max()

259082.36